In [1]:
import numpy as np
import pandas as pd
from IPython.display import display 

### Training and test set
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import Imputer, StandardScaler

from imblearn.over_sampling import SMOTE

from xgboost.sklearn import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, LinearRegression
from sklearn.base import clone
from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from mlxtend.classifier import StackingClassifier
from mlens.visualization import corrmat
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
[MLENS] backend: threading


In [2]:
df = pd.read_csv('./all/health-diagnostics-train.csv')
test = pd.read_csv('./all/health-diagnostics-test.csv')

df.replace('#NULL!',np.nan, inplace=True)
df.iloc[:, 0:(len(df.columns)-1)] = df.select_dtypes(include='object').apply(pd.to_numeric)
df.dropna(inplace=True)
test.replace('#NULL!',np.nan, inplace=True)
test = test.select_dtypes(include='object').apply(pd.to_numeric)
imp = Imputer(strategy='most_frequent')
imp.fit(test)
F_test = imp.transform(test)

In [58]:
X = df.drop('target',axis=1)
F_test = pd.DataFrame(F_test, columns=X.columns)
NoFam_test = F_test.drop('fam-history',axis=1)
NoFam_X = X.drop('fam-history',axis=1)
X.reset_index(drop=True, inplace=True)
NoFam_X.reset_index(drop=True, inplace=True)
y=df['target']
y.reset_index(drop=True, inplace=True)

In [59]:
seed = 23
np.random.seed(seed)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

In [61]:
train_idx = X_train.index
test_idx = X_test.index

In [62]:
Fam_Xtrain, Fam_Xtest = NoFam_X.iloc[train_idx], NoFam_X.iloc[test_idx]

In [63]:
def get_models():
    lrc = LogisticRegression(C=10, class_weight='balanced', random_state=seed)
    
    sgd = SGDClassifier(loss='log', class_weight={0:0.01,1:0.99}, random_state=seed)
    
    rfc = RandomForestClassifier(
        n_estimators=40,
        max_depth=2,
        min_samples_leaf=0.003,
        class_weight={0:0.01,1:0.99},
        max_features='auto',
        random_state=seed
    )
    
    noFam_rfc = RandomForestClassifier(n_estimators=110, 
                           max_depth=2, 
                           min_samples_leaf=0.001, 
                           min_samples_split=0.03, 
                           class_weight={0: 0.01, 1: 0.99}, 
                           random_state=seed)
    
    models = {
        'lrc':lrc,
        'sgd':sgd,
        'rfc':rfc,
        'noFam':noFam_rfc
    }
    return models

In [64]:
def train_base_learners(base_learners, inp, fam_inp, out, verbose=True):
    """Train all base learners in the library."""
    if verbose: print("Fitting models.")
    for i, (name, m) in enumerate(base_learners.items()):
        if verbose: 
            print("%s..." % name, end=" ", flush=False)
        if name == 'noFam':
            m.fit(fam_inp, out)
        else:
            m.fit(inp, out)
        if verbose: print("done")

In [65]:
def predict_base_learners(pred_base_learners, inp, fam_inp, verbose=True):
    """Generate a prediction matrix."""
    P = np.zeros((inp.shape[0], len(pred_base_learners)))

    if verbose: print("Generating base learner predictions.")
    for i, (name, m) in enumerate(pred_base_learners.items()):
        if verbose: 
            print("%s..." % name, end=" ", flush=False)
        if name == 'noFam':
            p = m.predict_proba(fam_inp)
        else:
            p = m.predict_proba(inp)
        P[:, i] = p[:, 1]
        if verbose: print("done")
    return P

In [66]:
def ensemble_predict(base_learners, meta_learner, inp, fam_inp, verbose=True):
    """Generate predictions from the ensemble."""
    P_pred = predict_base_learners(base_learners, inp, fam_inp, verbose=verbose)
    return P_pred, meta_learner.predict(P_pred)

In [67]:
def stacking(base_learners, meta_learner, X_stack, Fam_Xstack, y_stack, generator):
    print("Fitting final base learners...", end="")
    train_base_learners(base_learners, X_stack, Fam_Xstack, y_stack, verbose=False)
    print("done")
    kf = KFold(n_splits=10, random_state=seed)
    cv_preds, cv_y = [], []
    for i, (train_idx, test_idx) in enumerate(kf.split(X_stack)):
        fold_xtrain, fold_ytrain = X_stack.iloc[train_idx, :], y_stack.iloc[train_idx]
        fam_xtrain, fam_ytrain = Fam_Xstack.iloc[train_idx, :], y_stack.iloc[train_idx]
        fold_xtest, fold_ytest = X_stack.iloc[test_idx, :], y_stack.iloc[test_idx]
        fam_xtest, fam_ytest = Fam_Xstack.iloc[test_idx, :], y_stack.iloc[test_idx]
        
        # Inner loop: step 4 and 5
        fold_base_learners = {name: clone(model) for name, model in base_learners.items()}
        train_base_learners(fold_base_learners, fold_xtrain, fam_xtrain, fold_ytrain, verbose=False)
        fold_P_base = predict_base_learners(fold_base_learners, fold_xtest, fam_xtest, verbose=False)

        cv_preds.append(fold_P_base)
        cv_y.append(fold_ytest)
        print("Fold %i done" % (i + 1))

    np.vstack(cv_preds)
    X_meta = np.vstack(cv_preds)
    y_meta = np.hstack(cv_y)
    meta_learner.fit(X_meta, y_meta)
    return base_learners, meta_learner

In [68]:
meta_rfc = RandomForestClassifier(
    n_estimators=25, 
    max_depth=4, 
    max_features=3, 
    min_impurity_decrease=0.02, 
    min_samples_leaf=0.003,
    min_samples_split=0.01,
    class_weight={0:0.01, 1:0.99})

In [69]:
weak_learners, meta_learner = stacking(get_models(), meta_rfc, X_train, Fam_Xtrain, y_train, KFold(n_splits=10, random_state=seed))

Fitting final base learners...

/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


done


/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Fold 1 done


...

Fold 8 done


/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Fold 9 done


/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Fold 10 done


In [70]:
P, y_pred = ensemble_predict(weak_learners, meta_learner, X_test, Fam_Xtest)

Generating base learner predictions.
lrc... done
sgd... done
rfc... done
noFam... done


In [71]:
roc_auc_score(y_test, y_pred)

0.858832467874321

In [72]:
pd.crosstab(y_test,y_pred)

col_0,0,1
target,,
0,7525,73
1,3,8


In [73]:
test = pd.DataFrame(P, columns=['lrc','sgd','rfc','noFam'])

In [74]:
actuals = pd.Series(y_test).reset_index(drop=True)

In [77]:
x=0.5
y=0.5
z=0.5
f=0.01
v=2
boundaries = {'lrc':x, 'sgd':y, 'rfc':z,'noFam':f}
votes = {}
for model in test.columns:
    votes[model] = np.where(test[model]>boundaries[model],1,0)

votes_df = pd.DataFrame(votes)
votes_df['total_votes'] = votes_df.sum(axis=1)
predictions = np.where(votes_df['total_votes']>v,1,0)
display(roc_auc_score(y_test,y_pred))
pd.crosstab(y_test,predictions)

0.858832467874321

col_0,0,1
target,,
0,7531,67
1,3,8


In [ ]:
np.sum(predictions)

In [ ]:
boundaries = {'lrc':0.8, 'sgd':0.6, 'rfc':0.5}
votes = {}
for model in ['lrc','sgd','rfc']:
    votes[model] = np.where(test[model]>boundaries[model],1,0)
votes_df = pd.DataFrame(votes)

In [ ]:
results = pd.concat([votes_df,test['noFam'],actuals],axis=1)

In [ ]:
results[(results['rfc']==0)&(results['target']==1)]